In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

In [2]:
sql = '''
SELECT *
FROM dividends
LIMIT 1'''
struct = pd.read_sql(sql, conpf)
struct.dtypes

id            int64
stock_id      int64
name         object
year          int64
quarter       int64
number        int64
ppu         float64
amt         float64
net         float64
x_date       object
p_date       object
dtype: object

In [3]:
sql = '''
SELECT YEAR(p_date) AS p_year, MONTH(p_date) AS p_month, name, year, quarter, 
number, ppu, amt, net, x_date, p_date
FROM dividends
'''
df = pd.read_sql(sql, conpf)
df.dtypes

p_year       int64
p_month      int64
name        object
year         int64
quarter      int64
number       int64
ppu        float64
amt        float64
net        float64
x_date      object
p_date      object
dtype: object

In [4]:
df['x_date'] = pd.to_datetime(df['x_date'])
df['p_date'] = pd.to_datetime(df['p_date'])
df.dtypes

p_year              int64
p_month             int64
name               object
year                int64
quarter             int64
number              int64
ppu               float64
amt               float64
net               float64
x_date     datetime64[ns]
p_date     datetime64[ns]
dtype: object

In [5]:
df_groupby_year = df.groupby('p_year')
type(df_groupby_year)

pandas.core.groupby.generic.DataFrameGroupBy

In [6]:
df_groupby_year.ngroups

7

In [7]:
df_groupby_year.size()

p_year
2017     30
2018     48
2019     51
2020     54
2021    106
2022     71
2023     54
dtype: int64

In [8]:
df_cur_yr = df_groupby_year.get_group(2023)
df_cur_yr.columns

Index(['p_year', 'p_month', 'name', 'year', 'quarter', 'number', 'ppu', 'amt',
       'net', 'x_date', 'p_date'],
      dtype='object')

In [9]:
output = df_cur_yr.groupby('p_month').net.sum()
output

p_month
3     58267.70
4     28512.00
5    168108.72
6     70133.00
8      1350.00
9    108470.20
Name: net, dtype: float64

In [10]:
data_path = "../data/"
file_name = 'tmp-file-of-dividends.csv'
data_file = data_path + file_name
output.to_csv(data_file, index=True)

In [11]:
df_cur_yr.net.sum()

434841.62

### Old method

In [12]:
mask = df.p_year == 2023

In [13]:
ttl_by_month = df[mask].groupby(['p_year','p_month','name'], as_index=True).agg(
    {
        'net':['sum','count'],
    }
)
ttl_by_month

net      
                             sum count
p_year p_month name                   
2023   3       CPNREIT   2185.20     1
               DIF       9126.00     1
               GVREIT    7128.00     1
               JASIF    29900.00     1
               TFFIF     1032.00     1
               WHAIR     5481.00     1
               WHART     3415.50     1
       4       ASK       5832.00     1
               BANPU    20250.00     1
               PTTGC     1350.00     1
               SCC       1080.00     1
       5       ASP       3510.00     1
               BCH       5400.00     1
               IVL       2592.00     1
               JMART     3564.00     1
               JMT       2230.20     1
               KCE       5760.00     1
               MCS      13650.00     1
               NER       8370.00     1
               ORI      23085.00     1
               RCL      74520.00     1
               SENA      7503.52     1
               SINGER    1404.00     1
               STA       4640.00     1
               SYNEX     7020.00     1
               TMT       4860.00     1
       6       CPNREIT  13338.00     1
               DIF       6615.00     1
               GVREIT    5265.00     1
               IVL       1620.00     1
               JASIF    29900.00     1
               TFFIF     2064.00     1
               WHAIR     6160.50     1
               WHART     5170.50     1
       8       SCC       1350.00     1
       9       AH         772.80     1
               AIMIRT    2007.00     1
               ASP       2160.00     1
               BCH       1350.00     1
               CPNREIT  15660.00     1
               DIF       6399.00     1
               GVREIT    7616.70     1
               IVL       1620.00     1
               JASIF    29900.00     1
               JMT       1285.20     1
               KCE       5520.00     1
               NER       1350.00     1
               ORI       6480.00     1
               RCL       6750.00     1
               SYNEX     1350.00     1
               TFFIF     2018.00     1
               TMT       4860.00     1
               WHAIR     6160.50     1
               WHART     5211.00     1

In [14]:
ttl_by_month.groupby(level='p_month').sum()

net      
               sum count
p_month                 
3         58267.70     7
4         28512.00     4
5        168108.72    15
6         70133.00     8
8          1350.00     1
9        108470.20    19

In [15]:
ttl_by_month.sum()

net  sum      434841.62
     count        54.00
dtype: float64

In [16]:
ttl_by_month.sort_values(['p_month','name'],ascending=[True,True])

net      
                             sum count
p_year p_month name                   
2023   3       CPNREIT   2185.20     1
               DIF       9126.00     1
               GVREIT    7128.00     1
               JASIF    29900.00     1
               TFFIF     1032.00     1
               WHAIR     5481.00     1
               WHART     3415.50     1
       4       ASK       5832.00     1
               BANPU    20250.00     1
               PTTGC     1350.00     1
               SCC       1080.00     1
       5       ASP       3510.00     1
               BCH       5400.00     1
               IVL       2592.00     1
               JMART     3564.00     1
               JMT       2230.20     1
               KCE       5760.00     1
               MCS      13650.00     1
               NER       8370.00     1
               ORI      23085.00     1
               RCL      74520.00     1
               SENA      7503.52     1
               SINGER    1404.00     1
               STA       4640.00     1
               SYNEX     7020.00     1
               TMT       4860.00     1
       6       CPNREIT  13338.00     1
               DIF       6615.00     1
               GVREIT    5265.00     1
               IVL       1620.00     1
               JASIF    29900.00     1
               TFFIF     2064.00     1
               WHAIR     6160.50     1
               WHART     5170.50     1
       8       SCC       1350.00     1
       9       AH         772.80     1
               AIMIRT    2007.00     1
               ASP       2160.00     1
               BCH       1350.00     1
               CPNREIT  15660.00     1
               DIF       6399.00     1
               GVREIT    7616.70     1
               IVL       1620.00     1
               JASIF    29900.00     1
               JMT       1285.20     1
               KCE       5520.00     1
               NER       1350.00     1
               ORI       6480.00     1
               RCL       6750.00     1
               SYNEX     1350.00     1
               TFFIF     2018.00     1
               TMT       4860.00     1
               WHAIR     6160.50     1
               WHART     5211.00     1

In [17]:
ttl_by_month['net'].sum()

sum      434841.62
count        54.00
dtype: float64

In [18]:
sql = '''
SELECT YEAR(p_date) AS pay_year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
GROUP BY pay_year
ORDER BY pay_year DESC'''
ttl_by_year = pd.read_sql(sql, conpf)
ttl_by_year

,pay_year,grs_amt,net_amt,refund
0,2023,468013.00,434841.62,33171.38
1,2022,708367.91,661087.04,47280.87
2,2021,752315.10,700874.50,51440.60
3,2020,616228.94,576030.84,40198.10
4,2019,552424.34,510266.46,42157.88
5,2018,351895.60,309725.58,42170.02
6,2017,245981.10,219514.23,26466.87


In [30]:
ttl_by_year.net_amt.sum()

3412340.27

In [19]:
sql = '''
SELECT name, COUNT(*) AS qtrs, SUM(net) AS ttl_net
FROM dividends
GROUP BY name
ORDER BY SUM(net) DESC'''
total = pd.read_sql(sql, conpf)
total.sample(10)

,name,qtrs,ttl_net
110,TKN,1,1530.00
64,CPALL,2,6390.00
4,CPNREIT,14,138062.30
33,PTTEP,4,19485.00
101,HMPRO,1,2340.00
55,SMPC,1,7740.00
107,SQ,1,1800.00
36,TPIPP,6,15960.47
95,LPF,2,2778.75
62,POPF,3,6669.00


### Summarized profit by name

In [20]:
sql = '''
SELECT name, sum(profit) AS profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
GROUP BY stocks.name
ORDER BY sum(profit) DESC LIMIT 10'''
profits_by_stock = pd.read_sql(sql, conpf)
profits_by_stock

,name,profit
0,CPNREIT,547131.19
1,THANI,237046.32
2,BEAUTY,192974.08
3,JMT,187296.16
4,MTC,147177.71
5,ASIAN,141685.61
6,TU,125061.53
7,SAWAD,115506.59
8,SIS,110986.74
9,SINGER,103973.22


In [21]:
sql = '''
SELECT YEAR(date) AS year, MONTH(date) AS month, profit
FROM sells 
WHERE YEAR(date) = 2023
ORDER BY YEAR(date) DESC, MONTH(date) DESC
'''
sells = pd.read_sql(sql, conpf)
sells

,year,month,profit
0,2023,9,2793.14
1,2023,9,1311.90
2,2023,8,4398.53
3,2023,8,6418.09
4,2023,8,1687.04
5,2023,8,2146.35
6,2023,8,1093.15
7,2023,8,1312.41
8,2023,8,532.66
9,2023,7,1480.86


In [22]:
grouped = sells.groupby(['year','month'])
grouped

In [23]:
profit_by_month = grouped['profit'].sum()
profit_by_month

year  month
2023  1       -81338.26
      2       -77891.90
      3        -7863.28
      5         3680.06
      6       -75562.92
      7         1379.68
      8        17588.23
      9         4105.04
Name: profit, dtype: float64

In [24]:
grouped.agg(['sum','mean', 'max', 'min','count'])

profit                                        
                 sum          mean       max       min count
year month                                                  
2023 1     -81338.26  -8133.826000  14372.89 -33691.45    10
     2     -77891.90 -12981.983333  -3749.18 -34467.49     6
     3      -7863.28  -2621.093333   4594.67 -14620.17     3
     5       3680.06   3680.060000   3680.06   3680.06     1
     6     -75562.92  -5397.351429   2857.81 -43758.16    14
     7       1379.68    344.920000   4514.72  -4689.38     4
     8      17588.23   2512.604286   6418.09    532.66     7
     9       4105.04   2052.520000   2793.14   1311.90     2

### Temporary process to calculate dividend portion of profit

In [25]:
year = 2023
quarter = 4

In [26]:
sql = """
SELECT name, year, quarter, aq_eps, ay_eps
FROM epss 
WHERE year = %s AND quarter = %s
"""
sql = sql % (year, quarter)
df_epss = pd.read_sql(sql, conlt)
df_epss.head()

,name,year,quarter,aq_eps,ay_eps
0,MC,2023,4,0,0


In [27]:
sql = '''
SELECT name, dividend
FROM dividend 
'''
df_dividend = pd.read_sql(sql, const)
df_dividend

,name,dividend
0,WHART,0.7663
1,QH,0.1500
2,ADVANC,8.2400
3,CPNREIT,0.8977
4,MCS,0.2000
...,...,...
65,GFPT,0.1000
66,PTTEP,9.2500
67,MC,0.7700
68,RCL,1.2500


In [28]:
df_merge = pd.merge(df_epss, df_dividend, on='name', how='inner')
df_merge['xxx_pct'] = df_merge.dividend / df_merge.aq_eps * 100
df_merge.set_index('name',inplace=True)
df_merge.sort_values(['xxx_pct'],ascending=[True])

,year,quarter,aq_eps,ay_eps,dividend,xxx_pct
name,,,,,,
MC,2023,4,0,0,0.77,inf


In [29]:
df_merge.loc['TFFIF']

KeyError: 'TFFIF'